In [20]:
# Add visualization code for balance curve or trades
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import talib
import plotly.express as px

In [21]:
def custom_trading_strategy(df, initial_balance=5000, period=20, mode=1, invert_signal=1, filter1=41, filter2=0, commission=0.0001):
    """
    Custom trading strategy implementation based on the provided TradingView logic.
    
    Args:
        df (pd.DataFrame): A dataframe with columns 'date', 'open', 'high', 'low', 'close', 'volume'.
        initial_balance (float): Starting capital for backtesting (changed to 5000).
        period (int): The period for calculating the summation (similar to SMA in Pine Script).
        mode (int): The trading mode (1, 2, 3, or 4) to determine entry and exit conditions.
        invert_signal (int): Signal inversion (1 for normal, -1 for inverse).
        filter1 (float): The first filter threshold for the green condition.
        filter2 (float): The second filter threshold for the red condition.
        commission (float): Commission per trade as a fraction of trade size (changed to 0.0001 for 0.01%).
    
    Returns:
        pd.DataFrame: A dataframe containing the original data with buy/sell signals, profits, and positions.
    """
    
    # Calculate body and bodyVar (difference between current and previous body)
    df['body'] = df['close'] - df['open']
    df['bodyVar'] = df['body'].diff()

    # SumVar is similar to a moving average of bodyVar
    df['sumVar'] = talib.SMA(df['bodyVar'], timeperiod=period)

    # Initialize green and red columns
    df['green'] = np.where(df['sumVar'] > filter1, df['sumVar'], np.nan)
    df['red'] = np.where(df['sumVar'] < -filter2, df['sumVar'], np.nan)

    # Initialize signals for c1 and c2 based on trading mode
    df['c1'] = np.nan
    df['c2'] = np.nan

    if mode == 1:
        df['c1'] = df['red'] < df['red'].shift(1)
        df['c2'] = df['green'] > df['green'].shift(1)
    elif mode == 2:
        df['c1'] = df['red'] > df['red'].shift(1)
        df['c2'] = df['green'] < df['green'].shift(1)
    elif mode == 3:
        df['c1'] = df['red'] < df['red'].shift(1)
        df['c2'] = df['green'] < df['green'].shift(1)
    elif mode == 4:
        df['c1'] = df['red'] > df['red'].shift(1)
        df['c2'] = df['green'] > df['green'].shift(1)

    # Generate buy/sell signals based on c1 and c2
    df['signal'] = np.nan
    df['signal'] = np.where(df['c1'], 1 * invert_signal, df['signal'])
    df['signal'] = np.where(df['c2'], -1 * invert_signal, df['signal'])

    # Initialize position and profit tracking columns
    df['position'] = 0
    df['entry_price'] = np.nan
    df['exit_price'] = np.nan
    df['profit'] = 0
    df['balance'] = initial_balance
    df['buy_signal'] = 0
    df['sell_signal'] = 0
    position_size = 0
    is_long = False
    is_short = False

    # Iterate through the DataFrame and apply the strategy
    for i in range(1, len(df)):
        if df['signal'].iloc[i-1] == 1 and not is_long:
            # Long entry
            df.at[i, 'position'] = 1
            df.at[i, 'entry_price'] = df['open'].iloc[i]  # Changed from close to open
            position_size = df['balance'].iloc[i-1] / df['open'].iloc[i]  # Buy shares
            is_long = True
            is_short = False
            df.at[i, 'balance'] = df['balance'].iloc[i-1] - (df['open'].iloc[i] * position_size * (1 + commission))
            df.at[i, 'buy_signal'] = 1

        elif df['signal'].iloc[i-1] == -1 and not is_short:
            # Short entry
            df.at[i, 'position'] = -1
            df.at[i, 'entry_price'] = df['open'].iloc[i]  # Changed from close to open
            position_size = df['balance'].iloc[i-1] / df['open'].iloc[i]  # Short shares
            is_short = True
            is_long = False
            df.at[i, 'balance'] = df['balance'].iloc[i-1] - (df['open'].iloc[i] * position_size * (1 + commission))
            df.at[i, 'sell_signal'] = 1

        elif df['position'].iloc[i-1] == 1 and is_long:
            # Exit long position if the signal turns negative
            if df['signal'].iloc[i] == -1 or (df['position'].iloc[i] == 0):
                df.at[i, 'exit_price'] = df['open'].iloc[i]  # Changed from close to open
                df.at[i, 'profit'] = (df['exit_price'].iloc[i] - df['entry_price'].iloc[i-1]) * position_size - (df['exit_price'].iloc[i] * position_size * commission * 2)  # Added commission for entry and exit
                df.at[i, 'balance'] = df['balance'].iloc[i-1] + df['profit'].iloc[i]
                position_size = 0
                is_long = False
                df.at[i, 'sell_signal'] = 1

        elif df['position'].iloc[i-1] == -1 and is_short:
            # Exit short position if the signal turns positive
            if df['signal'].iloc[i] == 1 or (df['position'].iloc[i] == 0):
                df.at[i, 'exit_price'] = df['open'].iloc[i]  # Changed from close to open
                df.at[i, 'profit'] = (df['entry_price'].iloc[i-1] - df['exit_price'].iloc[i]) * position_size - (df['exit_price'].iloc[i] * position_size * commission * 2)  # Added commission for entry and exit
                df.at[i, 'balance'] = df['balance'].iloc[i-1] + df['profit'].iloc[i]
                position_size = 0
                is_short = False
                df.at[i, 'buy_signal'] = 1

        # If no trade occurs, carry forward the balance
        if np.isnan(df['balance'].iloc[i]):
            df.at[i, 'balance'] = df['balance'].iloc[i-1]

    # Calculate cumulative balance
    df['balance'] = df['balance'].cumsum()

    return df

In [22]:
# Example usage:
# Load data into a dataframe (df should have columns: date, open, high, low, close, volume)
"""
data = {
    'date': pd.date_range(start='2023-01-01', periods=100, freq='D'),
    'open': np.random.rand(100) * 100,
    'high': np.random.rand(100) * 100 + 5,
    'low': np.random.rand(100) * 100 - 5,
    'close': np.random.rand(100) * 100,
    'volume': np.random.randint(1000, 5000, size=100)
}
df = pd.DataFrame(data)
"""

df = pd.read_csv('../NDX5years.csv')
df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

# Apply the custom trading strategy
df_with_signals = custom_trading_strategy(df)

# Save results
import os
path = os.getcwd()
df_with_signals.to_csv(f'{path}/NDX5years_signals.csv')


/tmp/ipykernel_22451/843006633.py:73: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/tmp/ipykernel_22451/843006633.py:90: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '53.99666224880476' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [23]:
df_with_signals

,date,open,high,low,close,volume,body,bodyVar,sumVar,green,...,c1,c2,signal,position,entry_price,exit_price,profit,balance,buy_signal,sell_signal
0,2019-10-25,7926.6001,8033.2900,7926.6001,8029.2202,1911580000,102.6201,NaN,NaN,NaN,...,False,False,NaN,0,NaN,NaN,0.0,5.000000e+03,0,0
1,2019-10-28,8071.3901,8119.7300,8070.8999,8110.6699,1959920000,39.2798,-63.3403,NaN,NaN,...,False,False,NaN,0,NaN,NaN,0.0,1.000000e+04,0,0
2,2019-10-29,8101.5498,8108.6099,8045.6401,8047.5098,1842720000,-54.0400,-93.3198,NaN,NaN,...,False,False,NaN,0,NaN,NaN,0.0,1.500000e+04,0,0
3,2019-10-30,8056.8999,8096.3301,8017.9199,8083.1099,1936060000,26.2100,80.2500,NaN,NaN,...,False,False,NaN,0,NaN,NaN,0.0,2.000000e+04,0,0
4,2019-10-31,8100.6201,8110.4399,8041.6499,8083.8301,2276140000,-16.7900,-43.0000,NaN,NaN,...,False,False,NaN,0,NaN,NaN,0.0,2.500000e+04,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1244,2024-10-07,19954.6602,19990.1191,19763.0391,19800.7402,5240320000,-153.9200,-163.7793,-11.744925,NaN,...,False,False,NaN,0,NaN,NaN,0.0,4.925999e+06,0,0
1245,2024-10-08,19902.2109,20132.6699,19881.0801,20107.7793,5566510000,205.5684,359.4884,4.928910,NaN,...,False,False,NaN,0,NaN,NaN,0.0,4.930999e+06,0,0
1246,2024-10-09,20097.5605,20284.4199,20047.8594,20268.8594,5228670000,171.2989,-34.2695,-10.000095,NaN,...,False,False,NaN,0,NaN,NaN,0.0,4.935999e+06,0,0
1247,2024-10-10,20165.2500,20314.2402,20117.1992,20241.7598,5855390000,76.5098,-94.7891,-5.507030,NaN,...,False,False,NaN,0,NaN,NaN,0.0,4.940999e+06,0,0


In [24]:
# Create subplots: candlestick chart and volume bars
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.03, subplot_titles=('NASDAQ', 'Balance Curve'), 
                    row_width=[0.7, 0.3])

# Add candlestick chart
fig.add_trace(go.Candlestick(
    x=df_with_signals['date'],
    open=df_with_signals['open'],
    high=df_with_signals['high'],
    low=df_with_signals['low'],
    close=df_with_signals['close'],
    name='OHLC'
), row=1, col=1)

# Add balance curve
fig.add_trace(go.Scatter(
    x=df_with_signals['date'],
    y=df_with_signals['balance'],
    mode='lines',
    name='Balance',
    line=dict(color='blue')
), row=2, col=1)

# Update layout
fig.update_layout(
    title='NASDAQ Trading Strategy with Balance Curve',
    xaxis_rangeslider_visible=False,
    height=800,
    showlegend=True,
    xaxis_title='Date',
    yaxis_title='Price',
    yaxis2_title='Balance'
)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'close': array([ 8029.2202,  8110.6699,  8047.5098, ..., 20268.8594, 20241.7598,
                              20271.9707]),
              'high': array([ 8033.29  ,  8119.73  ,  8108.6099, ..., 20284.4199, 20314.2402,
                             20313.1992]),
              'low': array([ 7926.6001,  8070.8999,  8045.6401, ..., 20047.8594, 20117.1992,
                            20142.1797]),
              'name': 'OHLC',
              'open': array([ 7926.6001,  8071.3901,  8101.5498, ..., 20097.5605, 20165.25  ,
                             20154.8398]),
              'type': 'candlestick',
              'x': array(['2019-10-25', '2019-10-28', '2019-10-29', ..., '2024-10-09',
                          '2024-10-10', '2024-10-11'], dtype=object),
              'xaxis': 'x',
              'yaxis': 'y'},
             {'line': {'color': 'blue'},
              'mode': 'lines',
              'name': 'Balance',
              'type': 'scatter',
              'x': array(['2019-10-25', '2019-10-28', '2019-10-29', ..., '2024-10-09',
                          '2024-10-10', '2024-10-11'], dtype=object),
              'xaxis': 'x2',
              'y': array([   5000.        ,   10000.        ,   15000.        , ...,
                          4935999.17008344, 4940999.17008344, 4945999.17008344]),
              'yaxis': 'y2'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'NASDAQ',
                                'x': 0.5,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Balance Curve',
                                'x': 0.5,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 0.6789999999999999,
                                'yanchor': 'bottom',
                                'yref': 'paper'}],
               'height': 800,
               'showlegend': True,
               'template': '...',
               'title': {'text': 'NASDAQ Trading Strategy with Balance Curve'},
               'xaxis': {'anchor': 'y',
                         'domain': [0.0, 1.0],
                         'matches': 'x2',
                         'rangeslider': {'visible': False},
                         'showticklabels': False,
                         'title': {'text': 'Date'}},
               'xaxis2': {'anchor': 'y2', 'domain': [0.0, 1.0]},
               'yaxis': {'anchor': 'x', 'domain': [0.709, 1.0], 'title': {'text': 'Price'}},
               'yaxis2': {'anchor': 'x2', 'domain': [0.0, 0.6789999999999999], 'title': {'text': 'Balance'}}}
})

In [25]:
commission = 0.001

In [26]:

# Prepare data for visualizations
df_with_signals['date'] = pd.to_datetime(df_with_signals['date'])
df_with_signals['profit_loss'] = df_with_signals['profit'].fillna(0)
df_with_signals['cumulative_profit_loss'] = df_with_signals['profit_loss'].cumsum()
df_with_signals['days_in_trade'] = df_with_signals['position'].groupby((df_with_signals['position'] != df_with_signals['position'].shift()).cumsum()).cumcount() + 1
df_with_signals['commission_paid'] = abs(df_with_signals['profit']) * commission

# %%
# Chart 1: Profit and Loss grouped by date
fig_pnl = px.bar(df_with_signals, x='date', y='profit_loss', title='Daily Profit and Loss')
fig_pnl.add_trace(go.Scatter(x=df_with_signals['date'], y=df_with_signals['cumulative_profit_loss'], 
                             mode='lines', name='Cumulative P&L'))
fig_pnl.update_layout(xaxis_title='Date', yaxis_title='Profit/Loss')


# %%
# Chart 2: Candlestick chart with buy/sell signals and exits
fig_signals = go.Figure(data=[go.Candlestick(x=df_with_signals['date'],
                open=df_with_signals['open'], high=df_with_signals['high'],
                low=df_with_signals['low'], close=df_with_signals['close'])])

# Add buy signals
fig_signals.add_trace(go.Scatter(
    x=df_with_signals.loc[df_with_signals['buy_signal'] == 1, 'date'],
    y=df_with_signals.loc[df_with_signals['buy_signal'] == 1, 'low'],
    mode='markers',
    marker=dict(symbol='triangle-up', size=10, color='green'),
    name='Buy Signal'
))

# Add sell signals
fig_signals.add_trace(go.Scatter(
    x=df_with_signals.loc[df_with_signals['sell_signal'] == 1, 'date'],
    y=df_with_signals.loc[df_with_signals['sell_signal'] == 1, 'high'],
    mode='markers',
    marker=dict(symbol='triangle-down', size=10, color='red'),
    name='Sell Signal'
))

# Add exit points
fig_signals.add_trace(go.Scatter(
    x=df_with_signals.loc[df_with_signals['exit_price'].notna(), 'date'],
    y=df_with_signals.loc[df_with_signals['exit_price'].notna(), 'exit_price'],
    mode='markers',
    marker=dict(symbol='circle', size=8, color='purple'),
    name='Exit Point'
))

fig_signals.update_layout(title='NASDAQ with Buy/Sell Signals and Exits',
                          xaxis_title='Date',
                          yaxis_title='Price')


# %%
# Chart 3: Commissions and Days in Order
fig_commission = make_subplots(specs=[[{"secondary_y": True}]])

fig_commission.add_trace(
    go.Bar(x=df_with_signals['date'], y=df_with_signals['commission_paid'], name="Commission Paid"),
    secondary_y=False,
)

fig_commission.add_trace(
    go.Scatter(x=df_with_signals['date'], y=df_with_signals['days_in_trade'], name="Days in Trade"),
    secondary_y=True,
)

fig_commission.update_layout(
    title_text="Commissions Paid and Days in Trade",
    xaxis_title="Date",
)

fig_commission.update_yaxes(title_text="Commission", secondary_y=False)
fig_commission.update_yaxes(title_text="Days in Trade", secondary_y=True)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'name': 'Commission Paid',
              'type': 'bar',
              'x': array([datetime.datetime(2019, 10, 25, 0, 0),
                          datetime.datetime(2019, 10, 28, 0, 0),
                          datetime.datetime(2019, 10, 29, 0, 0), ...,
                          datetime.datetime(2024, 10, 9, 0, 0),
                          datetime.datetime(2024, 10, 10, 0, 0),
                          datetime.datetime(2024, 10, 11, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([0., 0., 0., ..., 0., 0., 0.]),
              'yaxis': 'y'},
             {'name': 'Days in Trade',
              'type': 'scatter',
              'x': array([datetime.datetime(2019, 10, 25, 0, 0),
                          datetime.datetime(2019, 10, 28, 0, 0),
                          datetime.datetime(2019, 10, 29, 0, 0), ...,
                          datetime.datetime(2024, 10, 9, 0, 0),
                          datetime.datetime(2024, 10, 10, 0, 0),
                          datetime.datetime(2024, 10, 11, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([ 1,  2,  3, ..., 15, 16, 17]),
              'yaxis': 'y2'}],
    'layout': {'template': '...',
               'title': {'text': 'Commissions Paid and Days in Trade'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 0.94], 'title': {'text': 'Date'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Commission'}},
               'yaxis2': {'anchor': 'x', 'overlaying': 'y', 'side': 'right', 'title': {'text': 'Days in Trade'}}}
})

In [27]:
fig_signals



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'close': array([ 8029.2202,  8110.6699,  8047.5098, ..., 20268.8594, 20241.7598,
                              20271.9707]),
              'high': array([ 8033.29  ,  8119.73  ,  8108.6099, ..., 20284.4199, 20314.2402,
                             20313.1992]),
              'low': array([ 7926.6001,  8070.8999,  8045.6401, ..., 20047.8594, 20117.1992,
                            20142.1797]),
              'open': array([ 7926.6001,  8071.3901,  8101.5498, ..., 20097.5605, 20165.25  ,
                             20154.8398]),
              'type': 'candlestick',
              'x': array([datetime.datetime(2019, 10, 25, 0, 0),
                          datetime.datetime(2019, 10, 28, 0, 0),
                          datetime.datetime(2019, 10, 29, 0, 0), ...,
                          datetime.datetime(2024, 10, 9, 0, 0),
                          datetime.datetime(2024, 10, 10, 0, 0),
                          datetime.datetime(2024, 10, 11, 0, 0)], dtype=object)},
             {'marker': {'color': 'green', 'size': 10, 'symbol': 'triangle-up'},
              'mode': 'markers',
              'name': 'Buy Signal',
              'type': 'scatter',
              'x': array([datetime.datetime(2019, 12, 3, 0, 0),
                          datetime.datetime(2020, 1, 15, 0, 0),
                          datetime.datetime(2020, 1, 23, 0, 0),
                          datetime.datetime(2020, 2, 6, 0, 0),
                          datetime.datetime(2020, 2, 24, 0, 0),
                          datetime.datetime(2020, 2, 28, 0, 0),
                          datetime.datetime(2020, 3, 13, 0, 0),
                          datetime.datetime(2020, 4, 20, 0, 0),
                          datetime.datetime(2020, 5, 22, 0, 0),
                          datetime.datetime(2020, 5, 27, 0, 0),
                          datetime.datetime(2020, 6, 15, 0, 0),
                          datetime.datetime(2020, 7, 7, 0, 0),
                          datetime.datetime(2020, 7, 10, 0, 0),
                          datetime.datetime(2020, 8, 18, 0, 0),
                          datetime.datetime(2020, 8, 25, 0, 0),
                          datetime.datetime(2020, 9, 4, 0, 0),
                          datetime.datetime(2020, 9, 17, 0, 0),
                          datetime.datetime(2020, 10, 26, 0, 0),
                          datetime.datetime(2020, 11, 3, 0, 0),
                          datetime.datetime(2020, 11, 10, 0, 0),
                          datetime.datetime(2020, 12, 4, 0, 0),
                          datetime.datetime(2020, 12, 30, 0, 0),
                          datetime.datetime(2021, 1, 5, 0, 0),
                          datetime.datetime(2021, 1, 19, 0, 0),
                          datetime.datetime(2021, 1, 28, 0, 0),
                          datetime.datetime(2021, 2, 19, 0, 0),
                          datetime.datetime(2021, 2, 23, 0, 0),
                          datetime.datetime(2021, 3, 4, 0, 0),
                          datetime.datetime(2021, 3, 9, 0, 0),
                          datetime.datetime(2021, 3, 25, 0, 0),
                          datetime.datetime(2021, 4, 15, 0, 0),
                          datetime.datetime(2021, 4, 21, 0, 0),
                          datetime.datetime(2021, 4, 30, 0, 0),
                          datetime.datetime(2021, 5, 4, 0, 0),
                          datetime.datetime(2021, 5, 6, 0, 0),
                          datetime.datetime(2021, 5, 11, 0, 0),
                          datetime.datetime(2021, 6, 16, 0, 0),
                          datetime.datetime(2021, 6, 21, 0, 0),
                          datetime.datetime(2021, 6, 28, 0, 0),
                          datetime.datetime(2021, 7, 13, 0, 0),
                          datetime.datetime(2021, 7, 19, 0, 0),
                          datetime.datetime(2021, 8, 9, 0, 0),
                          datetime.datetime(2021, 8, 11, 0, 0),
                          datetime.datetime(2021, 8, 19, 0, 0),
  

In [28]:
fig_pnl

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'date=%{x}<br>profit_loss=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([datetime.datetime(2019, 10, 25, 0, 0),
                          datetime.datetime(2019, 10, 28, 0, 0),
                          datetime.datetime(2019, 10, 29, 0, 0), ...,
                          datetime.datetime(2024, 10, 9, 0, 0),
                          datetime.datetime(2024, 10, 10, 0, 0),
                          datetime.datetime(2024, 10, 11, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([0., 0., 0., ..., 0., 0., 0.]),
              'yaxis': 'y'},
             {'mode': 'lines',
              'name': 'Cumulative P&L',
              'type': 'scatter',
              'x': array([datetime.datetime(2019, 10, 25, 0, 0),
                          datetime.datetime(2019, 10, 28, 0, 0),
                          datetime.datetime(2019, 10, 29, 0, 0), ...,
                          datetime.datetime(2024, 10, 9, 0, 0),
                          datetime.datetime(2024, 10, 10, 0, 0),
                          datetime.datetime(2024, 10, 11, 0, 0)], dtype=object),
              'y': array([   0.        ,    0.        ,    0.        , ..., 1118.15261684,
                          1118.15261684, 1118.15261684])}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Daily Profit and Loss'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Date'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Profit/Loss'}}}
})